In [ ]:
import polars as pl

In [ ]:
restaurantes = pl.read_csv('/content/drive/MyDrive/Alura/restaurantes.csv', encoding='utf8')

In [ ]:
restaurantes.head()

In [ ]:
restaurantes.glimpse()

### Convertendo valores numéricos

In [ ]:
restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
)

In [ ]:
restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
    .cast(pl.Float64)
)

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
    .cast(pl.Float64)
)

### Coletando estatísticas

In [ ]:
restaurantes.select(
    pl.col('approx_cost(for two people)').max().alias('max'),
    pl.col('approx_cost(for two people)').min().alias('min'),
    pl.col('approx_cost(for two people)').mean().round(2).alias('média')
)

## Usando o Regex

In [ ]:
restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
)

In [ ]:
restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
    .cast(pl.Float64)
)


In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
    .cast(pl.Float64)
)

In [ ]:
restaurantes.select(
    pl.col('rate').max().alias('max'),
    pl.col('rate').min().alias('min')
)

Os valores obtidos revelam a amplitude das avaliações no mercado, desde os restaurantes mais bem avaliados até aqueles com pior reputação. Essa informação é crucial para entendermos o nível de exigência dos clientes e identificamos benchmarks de qualidade. Com essa base, podemos aprofundar a análise investigando a relação entre avaliações e outros fatores, como faixa de preço, tipo de restaurante e pratos mais populares, garantindo insights valiosos para o posicionamento da nova franquia.

## Convertendo textos para listas

Compreender as preferências gastronômicas dos clientes é essencial para definir o cardápio da nova franquia. Restaurantes bem-sucedidos geralmente possuem pratos que se destacam e atraem clientes recorrentes.<br>
Para garantir uma análise precisa, precisamos limpar e estruturar os dados das colunas `"dish_liked"` (pratos favoritos) e `"cuisines"` (tipos de culinária). Isso envolve dividir as listas de itens e remover espaços desnecessários, tornando os dados mais organizados para futuras análises.

In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .cast(pl.List(pl.String)),
    pl.col('cuisines')
    .cast(pl.List(pl.String))
)

In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(','),
    pl.col('cuisines')
    .str.split(',')
)

In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars()),
    pl.col('cuisines')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars()),
    pl.col('cuisines')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)